In [1]:
import pandas as pd
import numpy as np
from time import time
import json
from sqlalchemy import create_engine
import re
import datetime

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
plt.style.use('seaborn-whitegrid')
import numpy as np

In [2]:
# Set up the SQL connection
dbname = 'redshift'
url = 'panamera-ods.cnsuxis6zqxr.us-west-2.redshift.amazonaws.com:5439/panameraods'
engine = create_engine("postgresql://" + 'lucasdresl' + ":" + 'cQsY801a76x82x62aWDT' + "@" + url)

## Data extraction

In [3]:
query = """

-- Identificacion de gps
drop table if exists gps;
select
  meta_session_long,
  case
    when params_gps_on_off = 'true' then 'gps_prendido'
    when params_gps_on_off = 'false' then 'gps_apagado'
     else 'ni_idea' end as type_gps,
  params_lat,
  params_long
   into temp table gps
  from spectrum.hydra_p_olx_android
where    params_en = 'app_open'
  and year=2019
  and month=8
  and day=16
  and meta_session_long_count = 1
  and params_cc = 'in'
group by 1,2,3,4

-- Login complete por facebook o google
;drop table if exists gps;
select
  meta_session_long,
  case
    when params_gps_on_off = 'true' then 'gps_prendido'
    when params_gps_on_off = 'false' then 'gps_apagado'
     else 'ni_idea' end as type_gps,
  params_lat,
  params_long
   into temp table gps
  from spectrum.hydra_p_olx_android
where    params_en = 'app_open'
  and year=2019
  and month=8
  and day=16
  and meta_session_long_count = 1
  and params_cc = 'in'
group by 1,2,3,4


-- Indentificacion de que se logeo
;drop table if exists logging;
select
  meta_session_long,
  params_login_method
   into temp table logging
   from spectrum.hydra_p_olx_android
where
     params_en = 'login_sign_in_complete'
  and year=2019
  and month=8
  and day=16
  and meta_session_long_count = 1
  and params_cc = 'in'
group by 1,2


-- Indentificacion de usuario que skipeo el logging
;drop table if exists skip_logging;
select
  meta_session_long
   into temp table skip_logging
   from spectrum.hydra_p_olx_android
where
     params_en = 'onboarding_tap_not_now'
  and year=2019
  and month=8
  and day=16
  and meta_session_long_count = 1
  and params_cc = 'in'
group by 1


-- Usuarios que dieron deny
;drop table if exists deny_location;
select
  meta_session_long
   into temp table deny_location
   from spectrum.hydra_p_olx_android
where
     params_en = 'permissions_tap_deny'
  and params_permission_for = 'location'
  and year=2019
  and month=8
  and day=16
  and meta_session_long_count = 1
  and params_cc = 'in'
group by 1

;drop table if exists last_table;
select
  a.meta_session_long,
  a.type_gps,
  params_lat,
  params_long,
  case when d.meta_session_long is not null then 1 else 0 end as skipeo_logging,
  case when b.params_login_method is not null then 1 else 0 end as logeo,
  case when b.params_login_method is not null then b.params_login_method else 'no_log' end as login_method,
  case when c.meta_session_long is not null then 1 else 0 end as deny_location
  into temp table last_table
    from gps a
     left join logging b on a.meta_session_long=b.meta_session_long
     left join deny_location c on a.meta_session_long=c.meta_session_long
     left join skip_logging d on a.meta_session_long=d.meta_session_long
 group by 1,2,3,4,5,6;
 
 select * from last_table

"""


In [4]:
df = pd.read_sql_query(query.replace('%', '%%'), engine)


In [5]:
df.head()

,meta_session_long,type_gps,skipeo_logging,logeo,login_method,deny_location
0,16c98a2d7c7xa66bd73e,gps_prendido,0,1,phone,1
1,16c9a885389xe90c887f,gps_apagado,0,1,google,0
2,16c9af20b82xd2b7551e,gps_prendido,1,0,no_log,0
3,16c9b6eb8a4xd5f54e93,gps_apagado,1,1,phone,0
4,16c999a95aex469e3875,gps_prendido,0,1,google,0
